## Cleaning a messy data
https://oscarbaruffa.com/messy/

## IMPORT LIBRARIES

In [1]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install pycountry
!pip install emoji

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
import pycountry
import re
import emoji

## READING AND ACCESSING DATA

In [3]:
# Google Sheets CSV link
url = "https://docs.google.com/spreadsheets/d/1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw/export?format=csv&gid=1625408792"

# Load into DataFrame
df = pd.read_csv(url)

df.head()

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


## FIX HEADERS (IF NEEDED)

In [4]:
# Simplify column names
df = df.rename(columns={
    "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": "annual_salary"
})

In [5]:
df = df.rename(columns={
    "How old are you?": "age"
})

In [6]:
df = df.rename(columns={
    "How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.": "bonus"
})

In [7]:
df = df.rename(columns={
    "Please indicate the currency": "currency"
})

In [8]:
df = df.rename(columns={
    "What country do you work in?": "country"
})

In [9]:
df = df.rename(columns={
    "How many years of professional work experience do you have overall?": "years_working"
})

In [10]:
df = df.rename(columns={
    "What is your highest level of education completed?": "education"
})

In [11]:
df = df.rename(columns={
    "What is your gender?": "gender"
})

In [12]:
df = df.rename(columns={
    "What is your race? (Choose all that apply.)": "race"
})

In [13]:
df = df.rename(columns={
    "How many years of professional work experience do you have in your field?": "years_in_your_field"
})

In [14]:
df.head()

,Timestamp,age,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:",annual_salary,bonus,currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",country,"If you're in the U.S., what state do you work in?",What city do you work in?,years_working,years_in_your_field,education,gender,race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


### Select specific columns to be in the data frame to analyse

In [15]:
df=df[['Timestamp','age','annual_salary','bonus','currency','country','years_working','years_in_your_field','education','gender','race']]

In [16]:
df.head()

,Timestamp,age,annual_salary,bonus,currency,country,years_working,years_in_your_field,education,gender,race
0,4/27/2021 11:02:10,25-34,"55,000",0.0,USD,United States,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,"54,600",4000.0,GBP,United Kingdom,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"34,000",NaN,USD,US,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,"62,000",3000.0,USD,USA,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"60,000",7000.0,USD,US,8 - 10 years,5-7 years,College degree,Woman,White


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28149 entries, 0 to 28148
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Timestamp            28149 non-null  object 
 1   age                  28149 non-null  object 
 2   annual_salary        28149 non-null  object 
 3   bonus                20813 non-null  float64
 4   currency             28149 non-null  object 
 5   country              28149 non-null  object 
 6   years_working        28149 non-null  object 
 7   years_in_your_field  28149 non-null  object 
 8   education            27921 non-null  object 
 9   gender               27975 non-null  object 
 10  race                 27966 non-null  object 
dtypes: float64(1), object(10)
memory usage: 2.4+ MB


### Cleaning/modifying the datatype

In [18]:
# Extract year from Timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
df['year'] = df['Timestamp'].dt.year.astype('Int64')
df = df.drop(columns=['Timestamp'])

# Move 'year' column to the first position
cols = ['year'] + [col for col in df.columns if col != 'year']
df = df[cols]

In [19]:
df.head()

,year,age,annual_salary,bonus,currency,country,years_working,years_in_your_field,education,gender,race
0,2021,25-34,"55,000",0.0,USD,United States,5-7 years,5-7 years,Master's degree,Woman,White
1,2021,25-34,"54,600",4000.0,GBP,United Kingdom,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021,25-34,"34,000",NaN,USD,US,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021,25-34,"62,000",3000.0,USD,USA,8 - 10 years,5-7 years,College degree,Woman,White
4,2021,25-34,"60,000",7000.0,USD,US,8 - 10 years,5-7 years,College degree,Woman,White


In [20]:
# Age — normalize dash spacing
df['age'] = df['age'].str.strip().str.replace(r'\s*-\s*', '-', regex=True)

In [21]:
# Remove commas and convert to integer
df['annual_salary'] = df['annual_salary'].str.replace(',', '').astype(int)

# Bonus → integer (handle NaN)
df['bonus'] = df['bonus'].fillna(0).astype(int)

In [22]:
df.head()

,year,age,annual_salary,bonus,currency,country,years_working,years_in_your_field,education,gender,race
0,2021,25-34,55000,0,USD,United States,5-7 years,5-7 years,Master's degree,Woman,White
1,2021,25-34,54600,4000,GBP,United Kingdom,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021,25-34,34000,0,USD,US,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021,25-34,62000,3000,USD,USA,8 - 10 years,5-7 years,College degree,Woman,White
4,2021,25-34,60000,7000,USD,US,8 - 10 years,5-7 years,College degree,Woman,White


In [23]:
# Years fields — remove "years" and fix dash spacing
for col in ['years_working', 'years_in_your_field']:
    df[col] = (
        df[col]
          .str.replace('years', '', case=False, regex=False)
          .str.strip()
          .str.replace(r'\s*-\s*', '-', regex=True)
    )

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28149 entries, 0 to 28148
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   year                 28149 non-null  Int64 
 1   age                  28149 non-null  object
 2   annual_salary        28149 non-null  int64 
 3   bonus                28149 non-null  int64 
 4   currency             28149 non-null  object
 5   country              28149 non-null  object
 6   years_working        28149 non-null  object
 7   years_in_your_field  28149 non-null  object
 8   education            27921 non-null  object
 9   gender               27975 non-null  object
 10  race                 27966 non-null  object
dtypes: Int64(1), int64(2), object(8)
memory usage: 2.4+ MB


In [25]:
df.head()

,year,age,annual_salary,bonus,currency,country,years_working,years_in_your_field,education,gender,race
0,2021,25-34,55000,0,USD,United States,5-7,5-7,Master's degree,Woman,White
1,2021,25-34,54600,4000,GBP,United Kingdom,8-10,5-7,College degree,Non-binary,White
2,2021,25-34,34000,0,USD,US,2-4,2-4,College degree,Woman,White
3,2021,25-34,62000,3000,USD,USA,8-10,5-7,College degree,Woman,White
4,2021,25-34,60000,7000,USD,US,8-10,5-7,College degree,Woman,White


In [26]:
# Count different values in the 'age' column
df['age'].value_counts()

age
25-34         12690
35-44          9910
45-54          3194
18-24          1251
55-64           995
65 or over       95
under 18         14
Name: count, dtype: int64

In [27]:
# Count values in the 'currency' column
df['currency'].value_counts()

currency
USD        23442
CAD         1675
GBP         1593
EUR          650
AUD/NZD      505
Other        167
CHF           37
SEK           37
JPY           23
ZAR           16
HKD            4
Name: count, dtype: int64

In [28]:
# Count values in the 'country' column
df['country'].value_counts()

country
United States     9015
USA               7951
US                2614
Canada            1573
United States      668
                  ... 
 New Zealand         1
Cuba                 1
Australi             1
Cote d'Ivoire        1
Thailand             1
Name: count, Length: 386, dtype: int64

In [31]:
# Standard sovereign countries
standard_countries = [
    "Afghanistan","Argentina","Australia","Austria","Bangladesh","Belgium","Bosnia and Herzegovina",
    "Brazil","Bulgaria","Myanmar","Cambodia","Canada","Chile","China","Colombia","Congo","Costa Rica",
    "Côte d'Ivoire","Croatia","Cuba","Cyprus","Czech Republic","Denmark","Ecuador","Egypt","Eritrea",
    "Estonia","Finland","France","Germany","Ghana","Greece","Hungary","India","Indonesia","Ireland",
    "Israel","Italy","Jamaica","Japan","Jordan","Kenya","Kuwait","Latvia","Liechtenstein","Lithuania",
    "Luxembourg","Malaysia","Malta","Mexico","Morocco","Nigeria","Netherlands","New Zealand","Norway",
    "Pakistan","Panama","Philippines","Poland","Portugal","Qatar","Romania","Russia","Rwanda",
    "Saudi Arabia","Serbia","Sierra Leone","Singapore","Slovakia","Slovenia","Somalia","South Africa",
    "South Korea","Spain","Sri Lanka","Sweden","Switzerland","Taiwan","Tanzania","Thailand","Turkey",
    "Uganda","Ukraine","United Arab Emirates","United Kingdom","United States","Uruguay","Vietnam","Zimbabwe"
]

# Direct mapping for common messy variants (all lowercase keys)
direct_map = {
    "usa": "United States", "us": "United States", "u.s.": "United States",
    "u.s.a": "United States", "u.s.a.": "United States",
    "united states of america": "United States", "united states": "United States",
    "us of a": "United States", "america": "United States",

    "uk": "United Kingdom", "u.k.": "United Kingdom", "u.k": "United Kingdom",
    "england": "United Kingdom", "scotland": "United Kingdom",
    "wales": "United Kingdom", "northern ireland": "United Kingdom",

    "brasil": "Brazil",
    "czechia": "Czech Republic", "czech republic": "Czech Republic",
    "burma": "Myanmar",
    "france": "France",
    "new zealand aotearoa": "New Zealand",
    "new zealand": "New Zealand",
}


def clean_country_column(df, column, standard_countries, direct_map):
    cleaned = []
    standard_lower = {c.lower(): c for c in standard_countries}

    for value in df[column]:
        val_norm = str(value).strip().lower()

        # Case 1: already in standard list
        if val_norm in standard_lower:
            resolved = standard_lower[val_norm]

        # Case 2: direct mapping
        elif val_norm in direct_map:
            resolved = direct_map[val_norm]

        # Case 3: fuzzy suggestion
        else:
            guess, score = process.extractOne(value, standard_countries)
            print(f"\nFound '{value}' → Did you mean '{guess}'? (confidence {score}%)")
            choice = input("Accept suggestion? (y/n): ").strip().lower()

            if choice == "y":
                resolved = guess
            else:
                resolved = input("Enter the correct country: ").strip()

            # Store permanently in direct_map
            direct_map[val_norm] = resolved

        cleaned.append(resolved)
    
    df[column + "_cleaned"] = cleaned
    return df, direct_map


# Run interactive cleaner
df_cleaned, updated_map = clean_country_column(df, "country", standard_countries, direct_map)



Found 'The Netherlands' → Did you mean 'Netherlands'? (confidence 95%)


Accept suggestion? (y/n):  y



Found 'U.S>' → Did you mean 'United Arab Emirates'? (confidence 86%)


Accept suggestion? (y/n):  n
Enter the correct country:  United States



Found 'ISA' → Did you mean 'Bulgaria'? (confidence 72%)


...

Accept suggestion? (y/n):  y



Found 'Italia' → Did you mean 'Australia'? (confidence 75%)


Accept suggestion? (y/n):  n
Enter the correct country:  Italy



Found 'Hong KongKong' → Did you mean 'Congo'? (confidence 54%)


Accept suggestion? (y/n):  n
Enter the correct country:  China


In [32]:
df.head()

,year,age,annual_salary,bonus,currency,country,years_working,years_in_your_field,education,gender,race,country_cleaned
0,2021,25-34,55000,0,USD,United States,5-7,5-7,Master's degree,Woman,White,United States
1,2021,25-34,54600,4000,GBP,United Kingdom,8-10,5-7,College degree,Non-binary,White,United Kingdom
2,2021,25-34,34000,0,USD,US,2-4,2-4,College degree,Woman,White,United States
3,2021,25-34,62000,3000,USD,USA,8-10,5-7,College degree,Woman,White,United States
4,2021,25-34,60000,7000,USD,US,8-10,5-7,College degree,Woman,White,United States


In [33]:
df=df[['year','age','annual_salary','bonus','currency','country_cleaned','years_working','years_in_your_field','education','gender','race']]
df

,year,age,annual_salary,bonus,currency,country_cleaned,years_working,years_in_your_field,education,gender,race
0,2021,25-34,55000,0,USD,United States,5-7,5-7,Master's degree,Woman,White
1,2021,25-34,54600,4000,GBP,United Kingdom,8-10,5-7,College degree,Non-binary,White
2,2021,25-34,34000,0,USD,United States,2-4,2-4,College degree,Woman,White
3,2021,25-34,62000,3000,USD,United States,8-10,5-7,College degree,Woman,White
4,2021,25-34,60000,7000,USD,United States,8-10,5-7,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...
28144,2025,25-34,19200,10200,Other,Thailand,2-4,1 year or less,College degree,Woman,Asian or Asian American
28145,2025,25-34,25000,0,GBP,United Kingdom,2-4,2-4,College degree,Man,Another option not listed here or prefer not t...
28146,2025,18-24,8000,0,USD,United States,2-4,1 year or less,High School,Woman,White
28147,2025,25-34,1000000,30000,USD,India,5-7,5-7,College degree,Man,Asian or Asian American


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28149 entries, 0 to 28148
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   year                 28149 non-null  Int64 
 1   age                  28149 non-null  object
 2   annual_salary        28149 non-null  int64 
 3   bonus                28149 non-null  int64 
 4   currency             28149 non-null  object
 5   country_cleaned      28149 non-null  object
 6   years_working        28149 non-null  object
 7   years_in_your_field  28149 non-null  object
 8   education            27921 non-null  object
 9   gender               27975 non-null  object
 10  race                 27966 non-null  object
dtypes: Int64(1), int64(2), object(8)
memory usage: 2.4+ MB
